In [ ]:
##CNN of multilabel classification
##defined functions

##Utilty functions for loading data, plotting graphs.
import pandas as pd
import sklearn
from sklearn import preprocessing
import numpy as np
from sklearn.metrics import ConfusionMatrixDisplay, roc_curve, auc, confusion_matrix, classification_report, multilabel_confusion_matrix

from sklearn.preprocessing import label_binarize

import os
from pathlib import Path
import tensorflow as tf
import matplotlib.pyplot as plt
plt.rcParams['axes.unicode_minus'] = False
plt.style.use('fivethirtyeight')

import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import regularizers
import visualkeras ##may need to pip install this

import seaborn as sns
from itertools import cycle

def load_data():
    ##Load the CIFAR10 data
    (X_train,y_train),(X_test,y_test)=tf.keras.datasets.cifar10.load_data()
    ##Normalize the data
    X_train, X_test = X_train/255., X_test/255.

    return (X_train, y_train), (X_test, y_test)

def save_model(model):
    model_json = model.to_json()
    with open("model.json", "w") as json_file:
     json_file.write(model_json)
    #Save model weights
    model.save('model_one.h5')
    model.save_weights("model.weights.h5")
    print("Saved model to disk")
    return

###Only used while the model is running
def plotting_accuracy(history):
   # Plotting the Model Accuracy & Model Loss vs Epochs (Hidden Input)
    plt.figure(figsize=[20,8])

# summarize history for accuracy
    plt.subplot(1,2,1)
    plt.plot(history.history['accuracy'])
    plt.plot(history['val_accuracy'])
    plt.title('Model Accuracy', size=25, pad=20)
    plt.ylabel('Accuracy', size=15)
    plt.xlabel('Epoch', size=15)
    plt.legend(['train', 'test'], loc='upper left')
# summarize history for loss

    plt.subplot(1,2,2)
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    plt.title('Model Loss', size=25, pad=20)
    plt.ylabel('Loss', size=15)
    plt.xlabel('Epoch', size=15)
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    return

def plotting_predictions(model, X_test):
    #Checking the predictions! (Hidden Input)
    predictions = model.predict(X_test)

    plt.figure(figsize=[10,10])

    class_names = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

    plt.subplot(2,2,1)
    n = 3
    plt.imshow(X_test[n].reshape(32, 32, -1), cmap=plt.cm.binary)
    plt.title("Predicted value: " + str(class_names[np.argmax(predictions[n], axis=0)]), size=20)
    plt.grid(False)

    plt.subplot(2,2,2)
    n = 4
    plt.imshow(X_test[n].reshape(32, 32, -1), cmap=plt.cm.binary)
    plt.title("Predicted value: " + str(class_names[np.argmax(predictions[n], axis=0)]), size=20)
    plt.grid(False)

    plt.subplot(2,2,3)
    n = 8
    plt.imshow(X_test[n].reshape(32, 32, -1), cmap=plt.cm.binary)
    plt.title("Predicted value: " + str(class_names[np.argmax(predictions[n], axis=0)]), size=20)
    plt.grid(False)

    plt.subplot(2,2,4)
    n = 6
    plt.imshow(X_test[n].reshape(32, 32, -1), cmap=plt.cm.binary)
    plt.title("Predicted value: " + str(class_names[np.argmax(predictions[n], axis=0)]), size=20)
    plt.grid(False)

    plt.suptitle("Predictions of CIFAR-10 Data", size=30, color="#6166B3")

    plt.show()
##Multilabel Confusion Matrix
class_names = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']
def confusion_matrixA(model, y_test, X_test,class_names):
    y_hat = model.predict(X_test)
    y_pred = np.argmax(y_hat,axis=1)

    cm = confusion_matrix(y_test, y_pred)
    cm_percentage = cm.astype('float')/ cm.sum(axis=1)[:,np.newaxis]*100  ##Normalize the confusion matrix
    disp = ConfusionMatrixDisplay(confusion_matrix=cm_percentage, display_labels=class_names)
    fig, ax = plt.subplots(figsize=(10, 10))
    disp = disp.plot(xticks_rotation='vertical', ax=ax, cmap='Blues')
    plt.title("Confusion matrix")
    plt.show()
    return()

def learning_curves(history, epochs):
    pd.DataFrame(history.history).plot(
        figsize=(8,5),xlim=[0,epochs], ylim=[0,1],grid=True, xlabel='Epochs',
        style=['r--','r--.',"b-",'b-*'])
    plt.show()
    return()

def class_report(model,X_test, y_test, class_names):
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=-1)
    print(classification_report(y_test, y_pred, target_names=class_names))

    ##Plotting a ROC Curve
def ROC_curve(model,X_test, y_test,save_plot_path,denselayer,class_names):
    class_labels = class_names
    #model = load_model(model_file_path) # load model from filepath
    feature_extractor = tf.keras.Model(inputs = model.inputs, outputs = model.get_layer(f"{denselayer}").output) # extract dense output layer (will be softmax probabilities)
    y_score = feature_extractor.predict(X_test, batch_size = 64) # one hot encoded softmax predictions
    ytest_binary = label_binarize(y_test, classes = [0,1,2,3,4,5,6,7,8,9]) # one hot encode the test data true labels
    n_classes = y_score.shape[-1]

    fpr = dict()
    tpr = dict()
    roc_auc = dict()
# compute fpr and tpr with roc_curve from the ytest true labels to the scores
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(ytest_binary[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

# plot each class curve on single graph for multi-class one vs all classification
    colors = cycle(['blue', 'red', 'green', 'brown', 'purple', 'pink', 'orange', 'black', 'yellow', 'cyan'])
    for i, color, lbl in zip(range(n_classes), colors, class_labels):
            plt.plot(fpr[i], tpr[i], color = color, lw = 1.5,
            label = 'ROC Curve of class {0} (area = {1:0.3f})'.format(lbl, roc_auc[i]))
            plt.plot([0, 1], [0, 1], 'k--', lw = 1.5)
            plt.xlim([-0.05, 1.0])
            plt.ylim([0.0, 1.05])
            plt.xlabel('False Positive Rate')
            plt.ylabel('True Positive Rate')
            plt.title('ROC Curve for CIFAR-10 Multi-Class Data')
            plt.legend(loc = 'lower right', prop = {'size': 6})
            fullpath = save_plot_path.joinpath(save_plot_path.stem +'_roc_curve.png')
            plt.savefig(fullpath)
            plt.show()
    return()

def count_layers(model):
    layer_count = 0
    for layer in model.layers:
        layer_count +=1
    return layer_count





In [ ]:
os.getcwd()

'/content'

In [ ]:
!pip install visualkeras

In [ ]:
##CNN of multilabel classification
##Order of the packages is important
#pip install visualkeras  ## if needed
## Base model only CPU
##May need to set - export  CUDA_VISIBLE_DEVICES=""
print('Load Packages and run code')
import pandas as pd
import sklearn
from sklearn import preprocessing
import numpy as np
from sklearn.metrics import ConfusionMatrixDisplay, roc_curve, auc, confusion_matrix, classification_report, precision_score, recall_score, f1_score

from sklearn.preprocessing import label_binarize

import os
from pathlib import Path
import tensorflow as tf
import matplotlib.pyplot as plt
plt.rcParams['axes.unicode_minus'] = False
plt.style.use('fivethirtyeight')

import tensorflow as tf
print("Number of GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import regularizers
import visualkeras ##may need to pip install this

import seaborn
from itertools import cycle
########Importing functions from different files #####
#from utility_file_code import load_data,save_model, plotting_accuracy,plotting_predictions, ROC_curve
#from utility_file_code import confusion_matrixA, class_report, learning_curves


###############        Parameters      ################
#Defining the parameters, scaling and making categories

batch_size = 32
num_classess = 10
epochs =6
denselayer = 'dense_1'
class_names = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']
save_plot_path = Path(os.getcwd())


###Load the data

(X_train, y_train),(X_test,y_test) = load_data()

print(f"X_train.shape:', {X_train.shape}")
print("x_train =", type(X_train))
print(f"X_test.shape:', {X_test.shape}")

print(f"y_train.shape:', {y_train.shape}")
print(f'y type = ', type(y_train))
print(f"y_test.shape:', {y_test.shape}")

#### Adding classess to y

y_cat_train = to_categorical(y_train,num_classess)
y_cat_test = to_categorical(y_test, num_classess)

###Turning the data into batches

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_cat_train))
##Batch the training dataset

train_dataset = train_dataset.batch(batch_size)

#Shuffle and prefetch for better performance
train_dataset  = train_dataset.shuffle(buffer_size = 1000).prefetch(tf.data.AUTOTUNE)

##Develop model for CNN
model = Sequential()
model.add(Conv2D(64, (3,3), padding ='same',input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3,3), padding ='same',input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, kernel_regularizer=regularizers.L2(0.01)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classess))
model.add(Activation('softmax'))


#model.summary()

##Visualize the model
#visualkeras.layered_view(model, scale_xy=10, legend=True).show()

model.compile(loss='categorical_crossentropy',
optimizer='sgd',
metrics=['accuracy'])

# Training the model
history = model.fit(train_dataset,
            epochs=epochs,
            validation_data =  (X_test, y_cat_test))

save_model(model)
learning_curves(history, epochs)
class_report(model, X_test, y_test,class_names)
##Confustion matrix
confusion_matrixA(model, X_test, y_test, class_names)

plotting_accuracy(history)
plotting_predictions(model, X_test)
### ROC curve
save_plot_path = Path(os.getcwd())
#denselayer = 'dense_'+f'{epochs}'
print(denselayer)
ROC_curve(model,X_test, y_test,save_plot_path,denselayer)


print('Finish')



Load Packages and run code
Number of GPUs Available:  0
170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
X_train.shape:', (50000, 32, 32, 3)
x_train = <class 'numpy.ndarray'>
X_test.shape:', (10000, 32, 32, 3)
y_train.shape:', (50000, 1)
y type =  <class 'numpy.ndarray'>
y_test.shape:', (10000, 1)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/6
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 801s 509ms/step - accuracy: 0.3455 - loss: 9.8873 - val_accuracy: 0.5249 - val_loss: 6.3365
Epoch 2/6
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 800s 508ms/step - accuracy: 0.5327 - loss: 5.6296 - val_accuracy: 0.6100 - val_loss: 3.8236
Epoch 3/6
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 784s 496ms/step - accuracy: 0.5956 - loss: 3.5007 - val_accuracy: 0.6709 - val_loss: 2.4710
Epoch 4/6
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 774s 495ms/step - accuracy: 0.6529 - loss: 2.3197 - val_accuracy: 0.6962 - val_loss: 1.7652
Epoch 5/6
1457/1563 ━━━━━━━━━━━━━━━━━━━━ 50s 476ms/step - accuracy: 0.6936 - loss: 1.6724

In [ ]:
!pip install visualkeras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.7/993.7 kB 10.9 MB/s eta 0:00:00
